In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import pandas as pd
import argparse                 
import sklearn 
import numpy as np 
import logging 
import sklearn.metrics as metrics 
import scikitplot as skplt
import matplotlib.pyplot as plt                                  
from argparse import ArgumentParser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset, random_split
import torch.nn.functional as F
from collections import Counter, defaultdict
import itertools
import os 
import copy
import warnings
from dataload import makedata
from classifier import ClassifierTrainer
from models import VQ_Classifier
import itertools
from data import load_data
warnings.filterwarnings("ignore", category=UserWarning)
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
torch.set_num_threads(32) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(911)

/home/smjo/anaconda3/envs/xai_timeseries/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
dataset = 'flat'
classification_model = '/home/hschung/xai/xai_timeseries/classification_models/flat_conv_nonoverlap_128_4/model_290.pt'
vqvae_model = "/home/hschung/xai/xai_timeseries/vqvae_models/flat_vqvae_nonoverlap_16_4/model_300.pt"
model_type = 'cnn'
num_quantizer = 4
batch_size = 1

In [33]:
os.environ["CUDA_VISIBLE_DEVICES"] = '5'
len_position=12
ds = load_data(dataset, task = 'classification')

train_size = int(0.9 * len(ds))
val_size = int(0.1 * len(ds))
test_size = len(ds) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(ds, [train_size, val_size, test_size])
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, pin_memory=True)

task is classification
dataset is flat
X shape:torch.Size([2000, 192]), y shape:torch.Size([2000, 2])


### FIND codebook combination ###

In [34]:
codebook_dict=defaultdict(set)         #position:코드북 조합(len=num_+quantizers)

conv_net = VQ_Classifier(
    num_classes = 2,
    vqvae_model = vqvae_model,
    positions =0,
    mask = 0,
    auc_classification = False,
    model_type = model_type,
    len_position=len_position
).to(device)

a = torch.load(classification_model)
conv_net.load_state_dict(a['model_state_dict'])

for param in conv_net.parameters():
    param.requires_grad = False

conv_net.eval()

prev_samples_0=[]
prev_samples_1=[]
with torch.no_grad():    
    score=0
    for i,(data, labels) in enumerate(test_loader):
        data = data.unsqueeze(1).float()
        labels = labels.type(torch.LongTensor)
        labels = torch.argmax(labels, dim=1)
        data, labels = data.to(device), labels.to(device)
        output, codebook_tokens, recon, input= conv_net(data)
        if labels==0:
            prev_samples_0.append(codebook_tokens)
        else:
            prev_samples_1.append(codebook_tokens)
        #첫번째 코드북에 따라 유니크한 조합이 생성되는지 확인
        for idx in range(len_position):
            codebook_dict[idx].add(codebook_tokens[0,idx,:])

In [35]:
conbt = {key:len(val) for key, val in codebook_dict.items()}
conbt

{}

In [36]:
codebook_tokens

tensor([[[ 28,  99,  36,  12,  12,  26,  25,  69],
         [ 77,  72,  69,  13,  51,  69,  62,  60],
         [ 54,  19,  13,  97,  54,  45,  94,  59],
         [ 28, 126, 126,  25,  73,  71,  33,  33],
         [ 42,  37, 115,  91,  16,  72,  79,  41],
         [ 42,  37, 115,  91,  16,  72,  79,  41],
         [ 42,  37, 115,  91,  16,  72,  79,  41],
         [ 42,  37, 115,  91,  16,  72,  79,  41],
         [ 42,  37, 115,  91,  16,  72,  79,  41],
         [ 42,  37, 115,  91,  16,  72,  79,  41],
         [ 42,  37, 115,  91,  16,  72,  79,  41],
         [ 42,  37, 115,  91,  16,  72,  79,  41]]], device='cuda:0')

### Perturb each position with codebook_dict and generate new samples###

In [68]:
def greedy_perturb(pos, prev_sample, codebook_dict):
    result = []
    for voluteer in codebook_dict[pos]:
        tmp = prev_sample.clone().detach()
        print(voluteer.shape)
        print(tmp.shape)
        tmp[0,pos,:] = voluteer
        result.append(tmp)
    return result

In [69]:

for idx, sample in enumerate(prev_samples_0):
    new_samples_0=defaultdict(list)
    
    for pos in range(len_position):
        tmp = greedy_perturb(pos,sample, codebook_dict)
        new_samples_0[sample].append(tmp)
    break
'''
    
for idx, sample in enumerate(prev_samples_1):
    new_samples_1=defaultdict(list)
    new_samples_1[sample].append(greedy_perturb(sample))
    '''

torch.Size([1, 12, 8])
torch.Size([1, 12, 8])


RuntimeError: expand(torch.cuda.LongTensor{[12, 8]}, size=[8]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

In [76]:
len(codebook_dict.values())

12